

# ΣΥΓΚΡΙΣΗ ΜΕΘΟΔΩΝ ΒΕΛΤΙΣΤΟΠΟΙΗΣΗΣ ΣΕ ΓΝΩΣΤΑ ΠΡΟΒΛΗΜΑΤΑ

 <h2 id="tocheading">Περιεχόμενα</h2>
 <div id="toc"></div>

In [44]:
get_ipython().run_cell_magic(u'javascript', u'', u"$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')")

<IPython.core.display.Javascript object>

## 1) Κλήση των μεθόδων από την SciPy <a class="anchor" id="first-bullet"></a>

In [3]:
# Εισαγωγή των συναρτήσεων minimize (για τις BFGS, CG, CG-Newton, Powell, Nelder-Mead) και της basinhopping
# για μια βελτιωμένη μορφή του stohastic annealing.
from scipy.optimize import minimize, basinhopping

In [4]:
# Εισαγωγή της βιβλιοθήκης numpy για την κλήση των βασικών μαθηματικών συναρτήσεων (sin, cos, etc).
import numpy as np
PI = np.pi

In [5]:
# Κλάση που θα χρειαστεί κατά την κλήση της συνάρτησης για Stochastic Annealing (basinhopping).
class MyTakeStep(object):
    def __init__(self, stepsize=0.5):
        self.stepsize = stepsize
    def __call__(self, x):
        s = self.stepsize
        x[0] += np.random.uniform(-2.*s, 2.*s)
        x[1:] += np.random.uniform(-s, s, x[1:].shape)
        return x

## 2) Οι συναρτήσεις προς βελτιστοποίηση:

In [6]:
import math as m
import scipy
def square(x): return x*x

<i>Οι ακόλουθες συναρτήσεις είναι εξειδικευμένες στο πρόβλημα της βελτιστοποίησης χωρίς περιορισμούς όπως αναφέρουν οι More et al.
 
 Από το εν λόγω paper οι αύξοντες αριθμοί των συναρτήσεων (μετά από ταξινόμηση) είναι οι ακόλουθοι: (3, 4, 5, 7, 9, 11, 12, 14, 16, 18, 20, 21, 22, 23, 24, 25, 26, 35), στο πλήθος δεκαοχτώ και αντιστοιχούν παρακάτω στα λατινικά γράμματα από το a μέχρι το r.
 
Πριν τον ορισμό της κάθε συνάρτησης δίνονται οι δοκιμασμένες τιμές της αρχικής τιμής του διανύσματος (x,y), η βέλτιστη τιμή και η τιμή της f(x,y) στο ακρότατο.</i>

### a) Συνάρτηση (badly scaled) του Powell (3) <a class="anchor" id="third-bullet"></a>

In [10]:
'''
x0 = (0,1)
x* = (1.098E-5, 9.106)
f(x*) = 0  
'''
f1a = lambda x: (10**4)*x[0]*x[1] - 1
f2a = lambda x: (m.exp(-x[0]) + m.exp(-x[1]) - 1.0001)
powell_func = lambda x: square(f1a(x)) + square(f2a(x))
powell_f = lambda x: powell_func([x[0],x[1]])

# Ορισμός της powell μαζί με τις μερικές παραγώγους της
def powell_f2(x):
    f = square((10**4)*x[0]*x[1] - 1) + square((m.exp(-x[0]) + m.exp(-x[1]) - 1.0001))
    df = np.zeros(2)
    df[0] = 2*((10**4)*x[0]*x[1] - 1)*(10**4)*x[1] - 2*m.exp(-x[0])
    df[1] = 2*((10**4)*x[0]*x[1] - 1)*(10**4)*x[0] - 2*m.exp(-x[1])
    return f,df

# Υπολογισμός της Jacobian
#JPowell = np.Jacobian(powell_f)

#### Υπολογισμοί για την συνάρτηση του Powell

##### BFGS

In [11]:
powBFGS = minimize(powell_f, (0,1), method = 'bfgs', tol =1e-8)
powBFGS

      fun: 8.500389456886437e-06
 hess_inv: array([[ 4.08641802e-08, -1.47664629e-02],
       [-1.47664629e-02,  5.38438843e+03]])
      jac: array([-2.01896883e+02, -6.90434900e-04])
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 332
      nit: 36
     njev: 80
   status: 2
  success: False
        x: array([1.62323656e-05, 6.14759392e+00])

##### Newton-CG

In [12]:
powNCG = minimize(powell_f2, [0,1], method = 'Newton-CG', tol =1e-8, jac=True)
powNCG

     fun: 0.00019246178758676506
     jac: array([-9.07108946e+02, -3.29068731e-02])
 message: "Warning: CG iterations didn't converge.  The Hessian is not positive definite."
    nfev: 85
    nhev: 0
     nit: 29
    njev: 338
  status: 3
 success: False
       x: array([2.34255102e-05, 4.26895046e+00])

##### CG

In [13]:
powCG = minimize(powell_f2, (0,1), method = 'CG', tol =1e-8, jac=True)
powCG

     fun: 0.135188173469304
     jac: array([-0.74336516, -0.73563323])
 message: 'Desired error not necessarily achieved due to precision loss.'
    nfev: 81
     nit: 1
    njev: 69
  status: 2
 success: False
       x: array([1.00006282e-04, 1.00000000e+00])

##### Powell

In [14]:
powPOW = minimize(powell_f, (0,1), method = 'Powell', tol =1e-8)
powPOW

   direc: array([[-1.00183080e-07,  6.43545979e-02],
       [-9.29425357e-08,  6.08972768e-02]])
     fun: 3.3498146595120214e-08
 message: 'Maximum number of function evaluations has been exceeded.'
    nfev: 2041
     nit: 51
  status: 1
 success: False
       x: array([1.22839125e-05, 8.14042687e+00])

##### Nelder-Mead

In [15]:
powNM = minimize(powell_f, (0,1), method = 'nelder-mead', tol =1e-8)
powNM

 final_simplex: (array([[1.17714307e-05, 8.49512936e+00],
       [1.17991888e-05, 8.47504727e+00],
       [1.18285673e-05, 8.45422234e+00]]), array([8.59453336e-09, 9.54568776e-09, 1.04145728e-08]))
           fun: 8.594533356831049e-09
       message: 'Maximum number of function evaluations has been exceeded.'
          nfev: 401
           nit: 222
        status: 1
       success: False
             x: array([1.17714307e-05, 8.49512936e+00])

##### Basin-Hopping

In [18]:
minimizer_kwargs = {"method":"L-BFGS-B", "jac":True}
x0pow = [0.0, 1.0]
mytakestep = MyTakeStep()
powBH= basinhopping(powell_f2, x0pow, minimizer_kwargs=minimizer_kwargs, niter=500, take_step=mytakestep)
powBH
#print("global minimum: x = [%.4f, %.4f], f(x0) = %.4f" % (powANN.x[0], powANN.x[1], powANN.fun))

                        fun: 4.347024572768168e-11
 lowest_optimization_result:       fun: 4.347024572768168e-11
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
      jac: array([-2.09982928e-04, -1.34444812e+00])
  message: 'ABNORMAL_TERMINATION_IN_LNSRCH'
     nfev: 44
      nit: 2
   status: 2
  success: False
        x: array([9.15791607e+00, 1.09195539e-05])
                    message: ['requested number of basinhopping iterations completed successfully']
      minimization_failures: 74
                       nfev: 13299
                        nit: 500
                          x: array([9.15791607e+00, 1.09195539e-05])

 <u><center><h4>Αποτελέσματα για την συνάρτηση Powell </h4></center></u>
 
 | Μέθοδος        | x*           | f(x*)  |Σημειώσεις| n iterations | f evaluations|
 | :-------------: |:----------------:| :-----:| :-----:| :-----:| :-----:|
 | BFGS      | (1.623E-05,   6.147E+00) | 8.500E-06 |Desired error not necessarily achieved due to precision loss. | 36|332 |
 | Newton-CG      | (2.342E-05,   4.269E+00)     |   1.925E-04 |CG iterations didn't converge.  The Hessian is not positive definite. |29 |85 |
 | CG | (1.000E-04,   1.000E+00)      |    1.352E-01 | Desired error not necessarily achieved due to precision loss.|1 |81 |
 |Powell | (1.228E-05, 8.140E+00) | 3.350E-08  |Maximum number of function evaluations has been exceeded. |51 |2041 |
 |Nelder-Mead|(1.177E-05, 8.495E+00) |8.594E-09 |Maximum number of function evaluations has been exceeded. |222 | 401|
 |Basin Hopping|(1.199E-05, 8.336E+00) |1.639E-08 |ABNORMAL_TERMINATION_IN_LNSRCH | 2|44 |
 |Αναμενόμενη τιμή|(1.098E-05, 9.106E+00) | 0.0|- | -|- |

### b) Συνάρτηση (badly scaled) του Brown (4)

In [19]:
''''
x0 = (1, 1)
x* = (10^6, 2E-6)
f(x*) = 0
'''
f1b = lambda x: x[0] - 10**6
f2b = lambda x: x[1] - 2*10**(-6)
f3b = lambda x: x[0]*x[1] - 2

brown_func = lambda x: square(f1b(x)) + square(f2b(x)) + square(f3b(x))

def brown_func2(x):
    f = brown_func(x)
    df = np.zeros(2)
    df[0] = 2*(x[0] - 10**6) + 0                     + 2*(x[0]*x[1] - 2)*x[1] 
    df[1] = 0                + 2*(x[1] - 2*10**(-6)) + 2*(x[0]*x[1] - 2)*x[0]
    return f,df

#### Υπολογισμοί για την συνάρτηση του Brown

##### BFGS

In [20]:
brownBFGS = minimize(brown_func, (1,1), method = 'bfgs', tol =1e-8)
brownBFGS

      fun: 5.4024409971684954e-05
 hess_inv: array([[ 4.99900127e-01, -6.02531925e-10],
       [-6.02531925e-10,  4.96376960e-13]])
      jac: array([-2.42767142e-03,  4.02743345e+02])
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 320
      nit: 15
     njev: 78
   status: 2
  success: False
        x: array([9.99999999e+05, 1.99275079e-06])

##### Newton-CG

In [21]:
brownNCG = minimize(brown_func2, (1,1), method = 'Newton-CG', tol =1e-8, jac=True)
brownNCG

     fun: 249999000001.0149
     jac: array([2.00000094e+00, 5.00000500e+11])
 message: "Warning: CG iterations didn't converge.  The Hessian is not positive definite."
    nfev: 3
    nhev: 0
     nit: 2
    njev: 89
  status: 3
 success: False
       x: array([5.00001000e+05, 3.99999182e-06])

##### CG

In [22]:
brownCG = minimize(brown_func2, (1,1), method = 'CG', tol =1e-8, jac=True)
brownCG

     fun: 1.972152263054323e-31
     jac: array([1.77635684e-21, 8.88178420e-10])
 message: 'Optimization terminated successfully.'
    nfev: 43
     nit: 15
    njev: 43
  status: 0
 success: True
       x: array([1.e+06, 2.e-06])

##### Powell

In [23]:
brownPowell = minimize(brown_func, (1,1), method = 'Powell', tol =1e-8)
brownPowell

   direc: array([[ 0.00000000e+00,  1.00000000e+00],
       [ 1.10629822e-05, -4.42518404e-17]])
     fun: 1.960730365729318e-23
 message: 'Optimization terminated successfully.'
    nfev: 117
     nit: 4
  status: 0
 success: True
       x: array([1.e+06, 2.e-06])

##### Nelder-Mead

In [24]:
brownNM = minimize(brown_func, (1,1), method = 'Nelder-Mead', tol =1e-8)
brownNM

 final_simplex: (array([[1.00000000e+06, 2.00000000e-06],
       [1.00000000e+06, 2.00000000e-06],
       [1.00000000e+06, 2.00000001e-06]]), array([7.03867554e-18, 2.02110001e-17, 9.64893108e-17]))
           fun: 7.038675538192397e-18
       message: 'Optimization terminated successfully.'
          nfev: 335
           nit: 177
        status: 0
       success: True
             x: array([1.e+06, 2.e-06])

##### Basin Hopping

In [25]:
minimizer_kwargs = {"method":"L-BFGS-B", "jac":True}
x0brow = [1.0, 1.0]
mytakestep = MyTakeStep()
brownBH = basinhopping(brown_func2, x0brow, minimizer_kwargs=minimizer_kwargs, niter=500, take_step=mytakestep)
brownBH

                        fun: 2.0615370909368328e-15
 lowest_optimization_result:       fun: 2.0615370909368328e-15
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 8.00936558e-08, -4.27918327e-02])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 25
      nit: 13
   status: 0
  success: True
        x: array([1.00000000e+06, 1.99999998e-06])
                    message: ['requested number of basinhopping iterations completed successfully']
      minimization_failures: 0
                       nfev: 2026
                        nit: 500
                          x: array([1.00000000e+06, 1.99999998e-06])

 <u><center><h4>Αποτελέσματα για την συνάρτηση Brown</h4></center></u>
 
 | Μέθοδος        | x*           | f(x*)  |Σημειώσεις| n iterations | f evaluations|
 | :-------------: |:----------------:| :-----:|:-----:|:-----:|:-----:|
 | BFGS      |(1.000E+06, 1.993E-06) | 5.402E-05 |Desired error not necessarily achieved due to precision loss.|15|320|
 | Newton-CG      |(5.000E+05, 4.000E-06)| <b>2.500E+11</b>|CG iterations didn't converge.  The Hessian is not positive definite.|0|3|
 | CG |(1.000E+06, 2.000E-06)|1.972E-31 |Optimization terminated successfully.|15|43|
 |Powell |(1.000E+06, 2.000E-06)| 1.961E-23 |Optimization terminated successfully.|4|117|
 |Nelder-Mead|(1.000Ε+06, 2.000Ε-06) |7.039Ε-18 |Optimization terminated successfully.|177|335|
 |Basin Hopping|(1.000E+06, 2.000E-06) |2.061E-15 |Success|13|25|
 |<b>Αναμενόμενη τιμή</b>|(1.0E+06, 2.0E-6) | 0.0|-|-|-|

### c) Συνάρτηση του Beale (5)

In [26]:
''''
x0 = (1,1)
x* = (3, 0.5)
f(x*) = 0
'''
const = [1.5, 2.25, 2.625]
f1c = lambda x: const[0] -x[0]*(1 - x[1]) 
f2c = lambda x: const[1] -x[0]*(1 - x[1]*x[1])
f3c = lambda x: const[2] -x[0]*(1 - x[1]*x[1]*x[1])

beal_func = lambda x: square(f1c(x)) + square(f2c(x)) + square(f3c(x))

def beal_func2(x):
    f = beal_func(x)
    df = np.zeros(2)
    df[0] = (2*(const[0] -x[0]*(1 - x[1]))*(-(1 - x[1]))
            +2*(const[1] -x[0]*(1 - x[1]*x[1]))*(-(1 - x[1]*x[1]))
            +2*(const[2] -x[0]*(1 - x[1]*x[1]*x[1]))*(-(1 - x[1]*x[1]*x[1])))
    
    df[1] = (2*(const[0] -x[0]*(1 - x[1]))*x[0]
            +2*(const[1] -x[0]*(1 - x[1]*x[1]))*(2*x[1])
            +2*(const[2] -x[0]*(1 - x[1]*x[1]*x[1]))*(3*x[1]*x[1]))
    return f,df

In [27]:
# Επειδή βλέπουμε πως κάποιες διαδικασίες επαναλαμβάνονται θα προσπαθήσουμενα διευκολύνουμε λίγο την ζωή μας.

In [28]:
gradientMethods = ['Newton-CG', 'CG']
nogradientMethods = ['bfgs', 'Powell', 'nelder-mead']

def basinalgo(func,x0):
    minimizer_kwargs = {"method":"L-BFGS-B", "jac":True}    
    mytakestep = MyTakeStep()
    res = basinhopping(func, x0, minimizer_kwargs=minimizer_kwargs, niter=500, take_step=mytakestep)
    print 'Basin-Hopping',res

#### Υπολογισμοί για την συνάρτηση του Beale

##### Newton-CG, CG και Basin-Hopping

In [29]:
for meth in gradientMethods:
    res = minimize(beal_func2, [1,1], method = meth, tol =1e-8, jac=True)    
    print meth, res, '\n\n'

basinalgo(beal_func2,[1,1])

Newton-CG      fun: 7.140426805286356
     jac: array([-0.  , 27.75])
 message: "Warning: CG iterations didn't converge.  The Hessian is not positive definite."
    nfev: 2
    nhev: 0
     nit: 1
    njev: 85
  status: 3
 success: False
       x: array([1.        , 0.59489031]) 


CG      fun: 0.0008717260814011488
     jac: array([-0.04243121, -0.02439545])
 message: 'Desired error not necessarily achieved due to precision loss.'
    nfev: 75
     nit: 6
    njev: 65
  status: 2
 success: False
       x: array([2.93125818, 0.48407468]) 


Basin-Hopping                         fun: 3.057531210858267e-16
 lowest_optimization_result:       fun: 3.057531210858267e-16
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
      jac: array([-1.72084288e-08, -3.47065415e-08])
  message: 'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 82
      nit: 31
   status: 0
  success: True
        x: array([2.99999996, 0.49999999])
                    message: ['requested number of basi

##### BFGS, Powell και Nelder-Mead

In [30]:
for meth in nogradientMethods:
    res = minimize(beal_func, [1,1], method = meth, tol =1e-8, jac=None)    
    print meth, res, '\n\n'

bfgs       fun: 1.976096216049981e-14
 hess_inv: array([[3.14203789, 0.77829902],
       [0.77829902, 0.21450585]])
      jac: array([-1.33902498e-10,  3.33249863e-10])
  message: 'Optimization terminated successfully.'
     nfev: 72
      nit: 16
     njev: 18
   status: 0
  success: True
        x: array([2.99999966, 0.49999991]) 


Powell    direc: array([[-4.92350654e-02, -1.49820463e-02],
       [ 4.75289660e-07,  5.92879797e-08]])
     fun: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 263
     nit: 7
  status: 0
 success: True
       x: array([3. , 0.5]) 


nelder-mead  final_simplex: (array([[3.        , 0.5       ],
       [2.99999999, 0.5       ],
       [3.00000001, 0.5       ]]), array([6.11427615e-18, 6.87326691e-18, 7.90120545e-18]))
           fun: 6.114276148670061e-18
       message: 'Optimization terminated successfully.'
          nfev: 162
           nit: 84
        status: 0
       success: True
             x: array([3. , 0.5]) 




 <u><center><h4>Αποτελέσματα για την συνάρτηση Beale</h4></center></u>
 
 | Μέθοδος        | x*           | f(x*)  |Σημειώσεις|n iterations | f evaluations|
 | :-------------: |:----------------:| :-----:|:-----:|:-----:|:-----:|
 | BFGS      | (2.99999966,  0.49999991) | 1.976E-14 |Optimization terminated successfully.|16|72|
 | Newton-CG      | (1.        ,  0.59489031)     |  <b>7.1404</b>|Iterations didn't converge.  The Hessian is not positive definite.|1|2|
 | CG | (2.93125818,  0.48407468)      |   8.717E-04 |Desired error not necessarily achieved due to precision loss.|6|75|
 |Powell | ( 3.0E+00,   5.0E-01) | 0.0E+00 |Optimization terminated successfully.|7|263|
 |Nelder-Mead|(3.0Ε+00,   5.0Ε-01) |6.114-18 |Optimization terminated successfully.|84|162|
 |Basin Hopping|(3., 0.5) |7.448E-18 |Success|15|18|
 |<b>Αναμενόμενη τιμή</b>|(3.0. 0.5) | 0.0|-|-|-|

### d) Συνάρτηση ελικοειδούς κοιλάδας (Helical valey) (7)

In [31]:
''''
x0 = (-1,0,0)
x* = (1, 0, 0)
f(x*) = 0
'''
def theta(x0,x1):
    if x0 > 0:
        return np.arctan(x1/x0)/PI
    if x0 < 0:
        return 0.5 + np.arctan(x1/x0)/PI
    
f1d = lambda x: 10*(x[2] - 10*theta(x[0],x[1]))
f2d = lambda x: 10*((x[0]**2 + x[1]**2)**(1/2) - 1.0)
f3d = lambda x: x[2]

def helic_func(x): return square(f1d(x)) + square(f2d(x)) + square(f3d(x))

def helic_func2(x):
    f = helic_func(x)
    df = np.zeros(3)
    df[0] = ( 200*(x[2] - 10*theta(x[0],x[1]))*(x[1]/(x[0]**2 + x[1]**2))
            + 20*((x[0]**2 + x[1]**2)**(1/2) - 1.0)*(0.5*((x[0]**2 + x[1]**2)**(-1/2))*2*x[0])
            + 0 )
    df[1] = ( 200*(x[2] - 10*theta(x[0],x[1]))*(-x[1]/(x[0]**2 + x[1]**2))
            + 20*((x[0]**2 + x[1]**2)**(1/2) - 1.0)*(0.5*((x[0]**2 + x[1]**2)**(-1/2))*2*x[1])        
            + 0 )
    df[2] = ( 20*(10*(x[2] - 10*theta(x[0],x[1])))
            + 0    
            + 1 )
    return f,df

#### Υπολογισμοί για την ελικοειδή συνάρτηση

##### Newton-CG, CG και Basin-Hopping

In [32]:
for meth in gradientMethods:
    res = minimize(helic_func2, [-1,0,0], method = meth, tol =1e-8, jac=True)    
    print meth, res, '\n\n'

basinalgo(helic_func2,[-1,0,0])

Newton-CG      fun: 24.952525
     jac: array([0.00000000e+00, 0.00000000e+00, 2.13162821e-14])
 message: 'Optimization terminated successfully.'
    nfev: 3
    nhev: 0
     nit: 2
    njev: 8
  status: 0
 success: True
       x: array([-1.   ,  0.   ,  4.995]) 


CG      fun: 24.763464656625718
     jac: array([ 0.        ,  0.        , -6.81478751])
 message: 'Desired error not necessarily achieved due to precision loss.'
    nfev: 117
     nit: 2
    njev: 105
  status: 2
 success: False
       x: array([-1.        ,  0.        ,  4.96092606]) 


Basin-Hopping                         fun: 1.5257511107467787e-07
 lowest_optimization_result:       fun: 1.5257511107467787e-07
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>
      jac: array([-3.48199217e-09,  3.48199217e-09,  1.00018658e+00])
  message: 'ABNORMAL_TERMINATION_IN_LNSRCH'
     nfev: 122
      nit: 14
   status: 2
  success: False
        x: array([ 6.58921517e+00, -8.10284991e-04, -3.90497227e-04])
               

##### BFGS, Powell και Nelder-Mead

In [33]:
for meth in nogradientMethods:
    try:        
        res = minimize(helic_func, [-1,0,0], method = meth, tol =1e-8)    
        print meth, res, '\n\n'
    except TypeError:
        print "TypeError στην μέθοδο", meth, '\n\n'

bfgs       fun: 5.617178766903007e-13
 hess_inv: array([[ 1.16152934e+08, -8.34964885e+05, -7.57437475e+03],
       [-8.34964885e+05,  6.00214326e+03,  5.44462234e+01],
       [-7.57437475e+03,  5.44462234e+01,  4.98836497e-01]])
      jac: array([-7.89983584e-16, -3.22290236e-09,  3.71940305e-07])
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 807
      nit: 69
     njev: 159
   status: 2
  success: False
        x: array([ 3.47437828e+02, -8.19813000e-05, -7.49255645e-07]) 


TypeError στην μέθοδο Powell 


nelder-mead  final_simplex: (array([[-2.66453526e-15,  7.91666667e-03,  3.33333333e-04],
       [-9.86864911e-16,  7.91666667e-03,  3.33333333e-04],
       [-4.51764826e-16,  7.91666667e-03,  3.33333333e-04],
       [-4.60536958e-17,  7.91666667e-03,  3.33333333e-04]]), array([1.12222222e-05, 1.12222222e-05, 1.12222222e-05, 1.12222222e-05]))
           fun: 1.1222222150797854e-05
       message: 'Optimization terminated successfully.'
       

 <u><center><h4>Αποτελέσματα για την συνάρτηση Helical valey </h4></center></u>
 
 | Μέθοδος        | x*           | f(x*)  | Σημειώσεις|n iterations | f evaluations|
 | :-------------: |:----------------:| :-----:|:------:|:------:|:------:|
 | BFGS      | (3.474Ε+02, -8.198Ε-05, -7.492Ε-07) |  5.617Ε-13 | Desired error not necessarily achieved due to precision loss. |69|807|
 | Newton-CG      | (-1.   ,  0.   , 4.995)     |  2.495Ε+01|Optimization terminated successfully.|2|3|
 | CG | (-1., 0. ,5.001)      |   2.501Ε+01  | Desired error not necessarily achieved due to precision loss.|1|15|
 |Powell | Αποτυγχάνει| Αποτυγχάνει|TypeError|-|-|
 |Nelder-Mead|(-2.664Ε-15, 7.917Ε-03, 3.333Ε-04) |1.122Ε-05| Optimization terminated successfully. |62|264|
 |Basin Hopping|(-4.353E-05,4.252E-01,      1.4264E-05) |7.807E-06 | ABNORMAL_TERMINATION_IN_LNSRCH |15|84
 |<b>Αναμενόμενη τιμή</b>|(1, 0, 0) | 0.000E+00| - |-|-|

### e) Συνάρτηση Rosenbrock

In [34]:
''''
x0 = (-1.2,1)
x* = (1, 1)
f(x*) = 0
'''

rosen = lambda x: scipy.optimize.rosen(x)
drosen = lambda x: scipy.optimize.rosen_der(x)

def rosen2(x):
    f = rosen(x)
    df = drosen(x)
    return f,df

#### Υπολογισμοί για την συνάρτηση Rosenbrock

##### Newton-CG, CG και Basin-Hopping

In [35]:
for meth in gradientMethods:
    res = minimize(rosen2, [-1.2,1], method = meth, tol =1e-8, jac=True)    
    print meth, res, '\n'    
basinalgo(rosen2,[-1.2,1])

Newton-CG      fun: 2.820436673169213e-16
     jac: array([-4.83533341e-06,  2.42961908e-06])
 message: 'Optimization terminated successfully.'
    nfev: 108
    nhev: 0
     nit: 88
    njev: 503
  status: 0
 success: True
       x: array([1.00000002, 1.00000003]) 

CG      fun: 3.108363291788037e-21
     jac: array([2.31330510e-11, 4.41424675e-11])
 message: 'Optimization terminated successfully.'
    nfev: 80
     nit: 37
    njev: 79
  status: 0
 success: True
       x: array([1., 1.]) 

Basin-Hopping                         fun: 2.974726413997047e-19
 lowest_optimization_result:       fun: 2.974726413997047e-19
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 4.76909312e-09, -1.84701143e-09])
  message: 'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 26
      nit: 20
   status: 0
  success: True
        x: array([1., 1.])
                    message: ['requested number of basinhopping iterations completed successfully']
      minimization_fa

##### BFGS, Powell και Nelder-Mead

In [36]:
for meth in nogradientMethods:
    N = len(nogradientMethods)        
    try:                
        res = minimize(rosen, [-1.2,1], method = meth, tol =1e-8)                    
        print meth, res, '\n\n'        
    except TypeError:
        print "TypeError στην μέθοδο", meth, '\n\n'

bfgs       fun: 2.005589939414498e-11
 hess_inv: array([[0.49758137, 0.99543508],
       [0.99543508, 1.99638311]])
      jac: array([ 2.00307475e-10, -9.59676782e-11])
  message: 'Optimization terminated successfully.'
     nfev: 164
      nit: 34
     njev: 41
   status: 0
  success: True
        x: array([0.99999552, 0.99999104]) 


Powell    direc: array([[1.04431143e-02, 2.21271461e-02],
       [1.22640359e-06, 2.34961998e-06]])
     fun: 9.023089641531086e-28
 message: 'Optimization terminated successfully.'
    nfev: 850
     nit: 23
  status: 0
 success: True
       x: array([1., 1.]) 


nelder-mead  final_simplex: (array([[1.        , 1.        ],
       [1.        , 1.        ],
       [1.        , 1.00000001]]), array([1.09908895e-18, 6.91811979e-18, 1.63410657e-17]))
           fun: 1.0990889519195732e-18
       message: 'Optimization terminated successfully.'
          nfev: 219
           nit: 117
        status: 0
       success: True
             x: array([1., 1.]) 




 <u><center><h4>Αποτελέσματα για την συνάρτηση Rosenbrock </h4></center></u>
 
 | Μέθοδος        | x*           | f(x*)  | Σημειώσεις|n iterations | f evaluations|
 | :-------------: |:----------------:| :-----:|:------:|:------:|:------:|
 | BFGS      | (0.99999552,  0.99999104) |  2.005Ε-11 | Optimization terminated successfully. |34|164|
 | Newton-CG      | (1.00000002,  1.00000003)     |  2.820Ε-16| Optimization terminated successfully.  |88|108|
 | CG | (-1.01703014,  1.07468158)      |   4.231E+00  | Desired error not necessarily achieved due to precision loss.|1|15|
 |Powell | (1.0,1.0)| 9.023E-28|Optimization terminated successfully.|23|850|
 |Nelder-Mead|(1.0,1.0) |1.099E-18| Optimization terminated successfully. |117|219|
 |Basin Hopping|(1.0,1.0) |1.160E-19 | Success |22|28|
 |<b>Αναμενόμενη τιμή</b>|(1, 1) | 0.000E+00| - |-|-|

### f) Συνάρτηση Wood (14)

In [37]:
''''
x0 = (-3,-1,-3,-1)
x* = (1,1,1,1) 
f(x*) = 0
'''
f1e = lambda x: (10*(x[1] - x[0]*x[0]))
f2e = lambda x: (1 - x[0])
f3e = lambda x: np.sqrt(90)*(x[3] - x[2]*x[2])
f4e = lambda x: (1 - x[2])
f5e = lambda x: np.sqrt(10)*(x[1] + x[3] - 2)
f6e = lambda x: (1/np.sqrt(10))*(x[1] - x[3])

wood_func = lambda x: (square(f1e(x)) + square(f2e(x)) + square(f3e(x)) 
                      + square(f4e(x)) + square(f5e(x)) + square(f6e(x)))

def wood_func2(x):
    f = wood_func(x)
    df=np.zeros(4)
    df[0] = 2*100*(x[1]-x[0]*x[0])*(-2*x[0]) - 2*(1-x[0])
    df[1] = 2*100*(x[1]-x[0]*x[0]) + 2*10*(x[1]+x[3]-2) + 0.1*2*(x[1]-x[3])
    df[2] = 2*90*(x[3]-x[2]*x[2])*(-2*x[2]) - 2*(1-x[2])
    df[3] = 2*90*(x[3]-x[2]*x[2]) + 2*10*(x[1]+x[3]-2) - 2*0.1*(x[1]-x[3])
    return f,df

#### Υπολογισμοί για την συνάρτηση Wood

##### Newton-CG, CG και Basin-Hopping

In [38]:
for meth in gradientMethods:
    res = minimize(wood_func2, [-3,-1,-3,-1], method = meth, tol =1e-8, jac=True)    
    print meth, res, '\n\n'   

basinalgo(rosen2,[-3,-1,-3,-1])

Newton-CG      fun: 1.06581656222382e-16
     jac: array([ 1.36193966e-06, -9.61642044e-07, -2.60666610e-06,  9.99349939e-07])
 message: 'Optimization terminated successfully.'
    nfev: 239
    nhev: 0
     nit: 203
    njev: 1743
  status: 0
 success: True
       x: array([1.00000001, 1.00000001, 0.99999999, 0.99999999]) 


CG      fun: 9.365649740319796e-23
     jac: array([ 1.26703315e-10, -1.09292242e-10, -2.13938645e-10,  6.11019013e-11])
 message: 'Optimization terminated successfully.'
    nfev: 141
     nit: 62
    njev: 141
  status: 0
 success: True
       x: array([1., 1., 1., 1.]) 


Basin-Hopping                         fun: 1.4732878867431036e-17
 lowest_optimization_result:       fun: 1.4732878867431036e-17
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 3.10136050e-09, -1.18862080e-08, -7.07028589e-08,  3.39994477e-08])
  message: 'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 24
      nit: 22
   status: 0
  success: True
     

##### BFGS, Powell και Nelder-Mead

In [39]:
for meth in nogradientMethods:
    N = len(nogradientMethods)        
    try:                
        res = minimize(wood_func, [-3,-1,-3,-1], method = meth, tol =1e-8)        
        print meth, res, '\n\n'
        
    except TypeError:
        print "TypeError στην μέθοδο", meth, '\n\n'

bfgs       fun: 5.752568124345873e-13
 hess_inv: array([[ 0.14320309,  0.28445882, -0.13563519, -0.27218656],
       [ 0.28445882,  0.57001854, -0.27182589, -0.54542434],
       [-0.13563519, -0.27182589,  0.14295523,  0.28411303],
       [-0.27218656, -0.54542434,  0.28411303,  0.57015097]])
      jac: array([-3.67230804e-09,  2.17617517e-09,  5.94566076e-09, -2.62172259e-09])
  message: 'Optimization terminated successfully.'
     nfev: 600
      nit: 87
     njev: 100
   status: 0
  success: True
        x: array([0.99999981, 0.99999964, 1.00000006, 1.00000014]) 


Powell    direc: array([[-1.96875244e-04, -3.78146933e-04,  1.80204391e-04,
         3.63400715e-04],
       [ 0.00000000e+00,  1.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 7.95824879e-03,  1.75922976e-02,  1.26029587e-02,
         3.66436008e-02],
       [-1.54141571e-06, -5.60760229e-06, -1.91237846e-05,
        -2.26290685e-05]])
     fun: 3.259735600016998e-21
 message: 'Optimization terminated 

 <u><center><h4>Αποτελέσματα για την συνάρτηση Wood </h4></center></u>
 
 | Μέθοδος        | x*           | f(x*)  | Σημειώσεις| n iterations | f evaluations|
 | :-------------: |:----------------:| :-----:|:------:|:---:|:---:|
 | BFGS      | (0.99999981,  0.99999964,  1.00000006,  1.00000014) |  5.752Ε-13 | Optimization terminated successfully. |87|600|
 | Newton-CG | (1.00000001,  1.00000001,  0.99999999,  0.99999999)|  5.752Ε-13| Optimization terminated successfully.|203|239|
 | CG | (1.0,1.0,1.0,1.0)      |   1.845Ε-17  | Optimization terminated successfully.|79|168|
 |Powell | (1.0,1.0,1.0,1.0)| 3.260Ε-21|Optimization terminated successfully.|23|1416|
 |Nelder-Mead|(1.0,1.0,1.0,1.0) |1.526Ε-16| Optimization terminated successfully.|387|655|
 |Basin Hopping|(1.0,1.0,1.0,1.0) |1.197Ε-17 | Success |23|26|
 |<b>Αναμενόμενη τιμή</b>|(1, 1,1,1) | 0.000E+00| -|-|-|

### g) Συνάρτηση των Freudestein και Roth (2)

In [40]:
''''
x0 = (0.5,-2)
x* = (5,4) 
f(x*) = 0
x' = (11.41..., -0.8968...)
f(x') = 48.9842...
'''
f1e = lambda x: -13 + x[0] + ((5-x[1])*x[1]-2)*x[1]
f2e = lambda x: -29 + x[0] + ((x[1]+1)*x[1]-14)*x[1]

fr_func = lambda x: (square(f1e(x)) + square(f2e(x)))

def fr_func2(x):
    f = fr_func(x)
    df=np.zeros(2)
    df[0] = 2*(-13 + x[0] + ((5-x[1])*x[1]-2)*x[1] + -29 + x[0] + ((x[1]+1)*x[1]-14)*x[1])
    df[1] = (2*(-13 + x[0] + ((5-x[1])*x[1]-2)*x[1])*(((5-x[1])*x[1]-2) + x[1]*(5-2*x[1])) + 
            2*(-29 + x[0] + ((x[1]+1)*x[1]-14)*x[1])*(((x[1]+1)*x[1]-14)+x[1]*(1 + 2*x[1])))
    return f,df

#### Υπολογισμοί για την συνάρτηση Freudestein και Roth

##### Newton-CG, CG και Basin-Hopping

In [41]:
for meth in gradientMethods:
    res = minimize(fr_func2, [0.5, -2], method = meth, tol =1e-8, jac=True)
    print meth, res, '\n\n'    
basinalgo(fr_func2,[0.5, -2])

Newton-CG      fun: 48.98425367924003
     jac: array([ 8.8257196e-07, -1.4047721e-05])
 message: 'Optimization terminated successfully.'
    nfev: 14
    nhev: 0
     nit: 11
    njev: 56
  status: 0
 success: True
       x: array([11.41277905, -0.89680525]) 


CG      fun: 48.98425367923999
     jac: array([-1.45765778e-07,  2.45844743e-06])
 message: 'Desired error not necessarily achieved due to precision loss.'
    nfev: 126
     nit: 14
    njev: 114
  status: 2
 success: False
       x: array([11.41277899, -0.89680525]) 


Basin-Hopping                         fun: 48.98425367924
 lowest_optimization_result:       fun: 48.98425367924
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
      jac: array([-1.40308817e-07,  2.46476435e-06])
  message: 'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 17
      nit: 16
   status: 0
  success: True
        x: array([11.41277899, -0.89680525])
                    message: ['requested number of basinhopping iterations com

##### BFGS, Powell και Nelder-Mead

In [42]:
for meth in nogradientMethods:
    try:        
        res = minimize(fr_func, [0.5, -2], method = meth, tol =1e-8)    
        print meth, res, '\n\n'
    except TypeError:
        print "TypeError στην μέθοδο", meth, '\n\n'

bfgs       fun: 48.984253679240055
 hess_inv: array([[1.4291669 , 0.08624278],
       [0.08624278, 0.00626799]])
      jac: array([0., 0.])
  message: 'Optimization terminated successfully.'
     nfev: 44
      nit: 10
     njev: 11
   status: 0
  success: True
        x: array([11.4127789 , -0.89680526]) 


Powell    direc: array([[10.86615593,  0.39827488],
       [ 0.3135128 ,  0.03120398]])
     fun: 48.98425367924001
 message: 'Optimization terminated successfully.'
    nfev: 180
     nit: 5
  status: 0
 success: True
       x: array([11.41277923, -0.89680524]) 


nelder-mead  final_simplex: (array([[11.41277892, -0.89680526],
       [11.41277893, -0.89680526],
       [11.41277893, -0.89680526]]), array([48.98425368, 48.98425368, 48.98425368]))
           fun: 48.98425367924001
       message: 'Optimization terminated successfully.'
          nfev: 172
           nit: 87
        status: 0
       success: True
             x: array([11.41277892, -0.89680526]) 




 <u><center><h4>Αποτελέσματα για την συνάρτηση των Freudestein και Roth</h4></center></u>
 
 | Μέθοδος        | x*           | f(x*)  | Σημειώσεις| n iterations | f evaluations|
 | :-------------: |:----------------:| :-----:|:------:|:---:|:---:|
 | BFGS      | (11.4127789 ,  -0.89680526) |  48.984253E+08 | Σφάλμα ακρίβειας|10|204|
 | Newton-CG | (11.41277905,  -0.89680525) |  48.984253E+08 | Επιτυχής τερματισμός|11|14|
 | CG | (7.94388211, -1.0549995)     |   54.677406E+08  | Σφάλμα ακρίβειας|4|29|
 |Powell | (11.41277923,  -0.89680524)| 48.984253E+08|Επιτυχής τερματισμός|5|180|
 |Nelder-Mead|(11.41277892,  -0.89680526) |48.984253E+09| Επιτυχής τερματισμός|87|172|
 |Basin Hopping|(11.41277894,  -0.89680526) |48.984253E+09 | Επιτυχής τερματισμός |17|18|
 |<b>Αναμενόμενη τιμή</b>|(11.41..., -0.8968...) | 48.9842| -|-|-|